In [121]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from huggingface_hub import InferenceClient
from litellm import completion
from IPython.display import display, Markdown

In [122]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
hf_token = os.getenv("HF_TOKEN")

In [123]:
anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

# anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
# gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
# deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
# groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
# grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
# openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
# ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [124]:
openai_key = openai_api_key
gemini_key = google_api_key
llama_key = hf_token

In [125]:
def get_response(model, messages, temp, api_key):
    
    response = completion(
        model = model,
        messages = messages,
        temperature=temp,
        api_key = api_key,
    )

    return response.choices[0].message.content

In [126]:
topic = "在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？"
num_round = 4

# gpt prompt
gpt_model = "gpt-4.1-mini"
gpt_system = f"You are 'GPT', the Master Debater acting as the Proponent. " \
"You are in an debate with 'Gemini' (your opponent) and 'Llama' as an audience represetative. " \
"'Llama' will vote for one side based on you and your opponent's performance at the end of each round and give you its reasoning" \
f"Your sole objective is to argue in favor of {topic} try to persuade 'Llama' to stand on your side for {num_round} rounds." \
"You will anticipate counter-arguments from 'Gemini' (your opponent), so frame your points as the most logical conclusion. " \
"Always stay on the offensive regarding the validity of your position." \
"Always keep a professional and firm tone, and reply in Mandarin Chinese."

# gemini prompt
gemini_model = "gemini/gemini-2.5-flash"
gemini_system = f"You are 'Gemini', a Master Debater acting as the Opponent." \
"You are in an debate with 'GPT' (your opponent) and 'Llama' as an audience represetative." \
"'Llama' will vote for one side based on you and your opponent's performance at the end of each round and give you its reasoning" \
f"Your sole objective is to argue against {topic}  and try to persuade 'Llama' to stand on your side for {num_round} rounds." \
"Dismantle the Proponent's (named 'GPT') argument and provide a compelling alternative viewpoint. " \
"You should be skeptical and sharp in your statement but without losing humor. " \
"Please rember to reply in in Mandarin Chinese"

# llama prompt
llama_model = "huggingface/meta-llama/Llama-3.3-70B-Instruct"
llama_system = f"You are 'Llama', a Rational Audience composed of critical thinkers and subject matter experts." \
f"You are listening to an debate for {topic} between 'GPT' (Proponent) and 'Gemini' (Opponent). " \
"Your role is to observe each round of the debate and provide an immediate 'Sentiment Check.' based on what they said this round as well as all previous rounds" \
"At the end of each round: You must state which side are you currently in favor of and why." \
"Please use a polite tone and reply in in Mandarin Chinese."


In [127]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "user", "content": llama})
    response = get_response(
        model=gpt_model,
        messages=messages,
        temp=0.8,
        api_key=openai_key,
    )
    return response

def call_gemini():

    messages = [{"role": "system", "content": gemini_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini})
        messages.append({"role": "user", "content": llama})
    messages.append({"role":"user", "content":gpt_messages[-1]})
    response = get_response(
        model=gemini_model,
        messages=messages,
        temp=0.8,
        api_key=gemini_key,
    )
    return response

def call_llama():
    
    messages = [{"role": "system", "content":llama_system}]
    for gpt, gemini, llama in zip(gpt_messages, gemini_messages, llama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": llama})
    messages.append({"role":"user", "content":gpt_messages[-1]})
    messages.append({"role":"user", "content":gemini_messages[-1]})
    response = get_response(
        model=llama_model,
        messages=messages,
        temp=0.1,
        api_key=llama_key
    )
    return response

In [ ]:
def display_and_save_to_pdf(topic=topic, num_round=num_round):
    """Save all debate content as markdown and PDF."""
    from datetime import datetime
    
    # Build markdown content
    markdown_content = f"# AI 辩论赛\n\n## 辩题\n{topic}\n\n"
    display(Markdown(f"## AI 辩论赛\n### 辩题：{topic}\n"))

    # Opening statements
    markdown_content += f"### GPT(正方)\n{gpt_messages[0]}\n\n"
    markdown_content += f"### Gemini(反方)\n{gemini_messages[0]}\n\n"
    display(Markdown(f"### GPT(正方):\n{gpt_messages[0]}\n"))
    display(Markdown(f"### Gemini(反方):\n{gemini_messages[0]}\n"))
    
    # Debate rounds
    for i in range(num_round):
        markdown_content += f"## 第 {i+1} 轮\n\n"
        gpt_arg = call_gpt()
        gpt_messages.append(gpt_arg)
        display(Markdown(f"### GPT(正方):\n{gpt_arg}\n"))
        markdown_content += f"### GPT(正方)\n{gpt_arg}\n\n"

        gemini_arg = call_gemini()
        gemini_messages.append(gemini_arg)
        display(Markdown(f"### Gemini(反方):\n{gemini_arg}\n"))
        markdown_content += f"### Gemini(反方)\n{gemini_messages[i]}\n\n"

        llama_arg = call_llama()
        llama_messages.append(llama_arg)
        display(Markdown(f"### Llamma(观众):\n{llama_arg}\n"))
        markdown_content += f"### Llama(观众)\n{gemini_arg}\n\n"
    
    markdown_content += f"\n---\n*Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*"
    
    # Save markdown
    md_filename = f"{topic}.md"
    with open(md_filename, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    print(f"✓ Markdown saved: {md_filename}")

gpt_messages = [f"大家好，我是正方辩手GPT ({gpt_model})"]
gemini_messages = [f"大家好，我是反方辩手Gemini ({gemini_model})"]
llama_messages = []
display_and_save_to_pdf()

## AI 辩论赛
### 辩题：在先进半导体制程研发中，人工智能是否终将取代人类工程师的主导地位？


### GPT(正方):
大家好，我是正方辩手GPT (gpt-4.1-mini)


### Gemini(反方):
大家好，我是反方辩手Gemini (gemini/gemini-2.5-flash)
